In [1]:
import pandas as pd 
import glob

In [2]:
weather=pd.read_csv('//datc//opschaler//weather_data//weather.csv',delimiter='\t',comment='#',parse_dates=['datetime'])
weather=weather.set_index(['datetime'])
weather.head()

,DD,DR,FF,FX,N,P,Q,RG,SQ,T,T10,TD,U,VV,WW
datetime,,,,,,,,,,,,,,,
2015-01-01 00:20:00,200.700000,0.0,4.970000,6.900,0.0,1033.900000,0.0,0.0,0.0,3.200000,2.300000,1.000000,85.0,3650.000000,10.0
2015-01-01 00:20:10,200.838333,0.0,4.967833,6.899,0.0,1033.906667,0.0,0.0,0.0,3.198333,2.296667,0.998333,85.0,3641.683333,10.0
2015-01-01 00:20:20,200.976667,0.0,4.965667,6.898,0.0,1033.913333,0.0,0.0,0.0,3.196667,2.293333,0.996667,85.0,3633.366667,10.0
2015-01-01 00:20:30,201.115000,0.0,4.963500,6.897,0.0,1033.920000,0.0,0.0,0.0,3.195000,2.290000,0.995000,85.0,3625.050000,10.0
2015-01-01 00:20:40,201.253333,0.0,4.961333,6.896,0.0,1033.926667,0.0,0.0,0.0,3.193333,2.286667,0.993333,85.0,3616.733333,10.0


In [3]:
#READING ALL EXCELS
path='/datc/opschaler/smartmeter_data'
filenames = glob.glob(path + "/*.csv")

In [4]:
dfs_smart=[]
dfs_gas=[]

for f in filenames:
    df = pd.read_csv(f,delimiter=';', parse_dates=['Timestamp','gasTimestamp'],header=0)
    
    df=df.rename(index=str,columns={"Timestamp":"datetime"})
    df=df.rename(index=str,columns={"gasTimestamp":"datetime"})
    
    smart=df.iloc[:,:7]
    gas=df.iloc[:,7:]
    
    smart=smart.set_index(['datetime'])
    gas=gas.set_index(['datetime'])
    
    dfs_smart.append(smart)
    dfs_gas.append(gas)

In [20]:
dfs_smart[3]

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn
datetime,,,,,,
2017-03-07 16:45:39,4108.425,0.0,4623.579,0.0,190.0,0.0
2017-03-07 16:45:49,4108.425,0.0,4623.579,0.0,191.0,0.0
2017-03-07 16:45:59,4108.426,0.0,4623.579,0.0,99.0,0.0
2017-03-07 16:46:09,4108.426,0.0,4623.579,0.0,101.0,0.0
2017-03-07 16:46:19,4108.426,0.0,4623.579,0.0,97.0,0.0
2017-03-07 16:46:29,4108.427,0.0,4623.579,0.0,98.0,0.0
2017-03-07 16:46:39,4108.427,0.0,4623.579,0.0,99.0,0.0
2017-03-07 16:46:49,4108.427,0.0,4623.579,0.0,98.0,0.0
2017-03-07 16:46:59,4108.428,0.0,4623.579,0.0,100.0,0.0


In [18]:
"""
NaN is a missing row()
"""

t = dfs_smart[3].resample('10s').mean()
t.head()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [15]:
dfs_smart_resampled = []

for f in dfs_smart:
    f = f.resample('10s').mean()
    dfs_smart_resampled.append(f)

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [ ]:
dfs_smart_resampled[0]